In [1]:
#%matplotlib qt
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D  # sometimes needed to register 3D
from medmnist import OrganMNIST3D

train_dataset = OrganMNIST3D(split='train', size=28, download=True)
trainx = []
trainy = []

test_dataset = OrganMNIST3D(split='test', size=28, download=True)
testx = []
testy = []

val_dataset = OrganMNIST3D(split='train', size=28, download=True)
valx = []
valy = []

for i in range(len(train_dataset)):
    trainx.append(train_dataset[i][0])
    trainy.append(train_dataset[i][1])

for i in range(len(test_dataset)):
    testx.append(test_dataset[i][0])
    testy.append(test_dataset[i][1])

for i in range(len(val_dataset)):
    valx.append(val_dataset[i][0])
    valy.append(val_dataset[i][1])


trainx_tensor = tf.convert_to_tensor(trainx, dtype=tf.float16)
trainy_tensor = tf.convert_to_tensor(trainy, dtype=tf.float16)
testx_tensor = tf.convert_to_tensor(testx, dtype=tf.float16)
testy_tensor = tf.convert_to_tensor(testy, dtype=tf.float16)
valx_tensor = tf.convert_to_tensor(valx, dtype=tf.float16)
valy_tensor = tf.convert_to_tensor(valy, dtype=tf.float16)
# float16 doesn't run any faster on the 4090s, but it cuts memory usage in half!


In [2]:
def myNet():

    model = tf.keras.Sequential(layers = [
        # #images only have 1 color scale (greyscale)
        tf.keras.layers.InputLayer(shape=(1, 28, 28, 28)),

        #(3, 3, 3) slides a 3x3x3 cube over the height, depth, width
        # Using data_format channels first because the input has the channels (axis here) first instead of last
        tf.keras.layers.Conv3D(32, (3,3,3), activation= 'relu', data_format='channels_first'),
        tf.keras.layers.BatchNormalization(), #makes net less confident early on but more stable, it stabalizes the gradients
        tf.keras.layers.Conv3D(64, (2,2,2), activation= 'relu', data_format='channels_first'),
        tf.keras.layers.MaxPooling3D(pool_size=(2,2,2), data_format='channels_first'),  #pool size of (2, 2, 2) changes the input shape of (28, 28, 28, 1) to (14, 14, 14, 1)
        tf.keras.layers.Dropout(0.15),

        tf.keras.layers.Conv3D(128, (3,3,3), activation= 'relu', data_format='channels_first'),
        tf.keras.layers.BatchNormalization(), #drops the loss a fair bit
        tf.keras.layers.MaxPooling3D(pool_size=(2,2,2),data_format='channels_first'),

        #GlobalAveragePooling3D replaces flatten, hugely increases the accuracy (85%->92%) while also massively dropping the loss (0.9 -> 0.2)
        tf.keras.layers.GlobalAveragePooling3D(data_format='channels_first'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Dense(11, activation='softmax', dtype='float32', name= 'output') #needs to be 11 because there are 10 classifications
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [3]:
class saveNet(tf.keras.callbacks.Callback):
    def __init__(self, n):
        super().__init__()
        self.save_rate = n

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_rate == 0:
            filename = f'checkpoints/epoch_{epoch+1}.weights.h5'
            self.model.save_weights(filename, overwrite=True)
            print(f"Model weights saved to {filename}")

checkpoint = saveNet(30)
model = myNet()

training_history = model.fit(
    trainx_tensor, trainy_tensor,
    batch_size= 16,
    epochs= 3,
    callbacks = [checkpoint]
    )


Epoch 1/3
61/61 ━━━━━━━━━━━━━━━━━━━━ 43s 661ms/step - accuracy: 0.1874 - loss: 2.2035
Epoch 2/3
61/61 ━━━━━━━━━━━━━━━━━━━━ 39s 633ms/step - accuracy: 0.3522 - loss: 1.7053
Epoch 3/3
61/61 ━━━━━━━━━━━━━━━━━━━━ 39s 641ms/step - accuracy: 0.4387 - loss: 1.4416


In [4]:
test_loss, test_acc = model.evaluate(testx_tensor, testy_tensor)
print(f"Test Accuracy: {round(test_acc * 100, 2)}%")

20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 281ms/step - accuracy: 0.1082 - loss: 2.4908
Test Accuracy: 10.82%


In [6]:
trainedModel = myNet()
trainedModel.load_weights('checkpoints/epoch_60.weights.h5') 
print("Model loaded successfully from epoch_60.weights.h5")


# Trained model accuracy on 60 epochs
test_loss, test_acc = trainedModel.evaluate(testx_tensor, testy_tensor, verbose=0)
print(f"Test Accuracy: {test_acc * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

Model loaded successfully from epoch_60.weights.h5
Test Accuracy: 93.93%
Test Loss: 0.1875
